<a href="https://colab.research.google.com/github/jduell12/DS-Unit-2-Linear-Models/blob/master/C_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

In [2]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'

# If you're working locally:
else:
    DATA_PATH = '../data/'

# Module Project: Regression II

In this project, you'll continue working with the New York City rent dataset you used in the last module project.

## Directions

The tasks for this project are as follows:

- **Task 1:** Import `csv` file using `wrangle` function.
- **Task 2:** Conduct exploratory data analysis (EDA), and modify `wrangle` function to engineer two new features.
- **Task 3:** Split data into feature matrix `X` and target vector `y`.
- **Task 4:** Split feature matrix `X` and target vector `y` into training and test sets.
- **Task 5:** Establish the baseline mean absolute error for your dataset.
- **Task 6:** Build and train a `Linearregression` model.
- **Task 7:** Calculate the training and test mean absolute error for your model.
- **Task 8:** Calculate the training and test $R^2$ score for your model.
- **Stretch Goal:** Determine the three most important features for your linear regression model.

**Note**

You should limit yourself to the following libraries for this project:

- `matplotlib`
- `numpy`
- `pandas`
- `sklearn`

# I. Wrangle Data

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

# Libraries for R^2 visualization
from ipywidgets import interactive, IntSlider, FloatSlider
from math import floor, ceil
from sklearn.base import BaseEstimator, RegressorMixin


In [4]:
def checkPetNumber(cell_contents):
  if (cell_contents > 0):
    cell_contents = cell_contents - 1
  return cell_contents

def checkTruthValue(cell_contents):
  if(cell_contents > 0):
    cell_contents = 1
  return cell_contents

def wrangle(filepath):
    df = pd.read_csv(filepath, parse_dates=['created'], index_col='created')
    
    # Remove the most extreme 1% prices,
    # the most extreme .1% latitudes, &
    # the most extreme .1% longitudes
    df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
            (df['price'] <= np.percentile(df['price'], 99.5)) & 
            (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
            (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
            (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
            (df['longitude'] <= np.percentile(df['longitude'], 99.95))]
    df = df.dropna()
    #create new column for combination of cats and dogs allowed
    df['cat_and_dog_allowed'] = df['cats_allowed'] + df['dogs_allowed']
    df['cat_and_dog_allowed'] = df['cat_and_dog_allowed'].apply(checkPetNumber)
    #create new column for cat or dog allowed 
    df['cat_or_dog_allowed'] = df['cats_allowed'] + df['dogs_allowed']
    df['cat_or_dog_allowed'] = df['cat_or_dog_allowed'].apply(checkTruthValue)
    #drop unneeded columns 
    df = df.drop(columns=['description', 'display_address', 'street_address', 'interest_level',])
  
    return df


filepath = DATA_PATH + 'apartments/renthop-nyc.csv'

**Task 1:** Add the following functionality to the above `wrangle` function.

- The `'created'` column will parsed as a `DateTime` object and set as the `index` of the DataFrame. 
- Rows with `NaN` values will be dropped.

Then use your modified function to import the `renthop-nyc.csv` file into a DataFrame named `df`.

In [5]:
df = wrangle(filepath)

df.head()

,bathrooms,bedrooms,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,cat_and_dog_allowed,cat_or_dog_allowed
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,40.7145,-73.9425,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-12 12:19:27,1.0,2,40.7947,-73.9667,5465,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
2016-04-17 03:26:41,1.0,1,40.7388,-74.0018,2850,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-18 02:22:02,1.0,1,40.7539,-73.9677,3275,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-28 01:32:41,1.0,4,40.8241,-73.9493,3350,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
df.shape

(47260, 31)

In [7]:
df['cat_and_dog_allowed'].value_counts()

0    26336
1    20924
Name: cat_and_dog_allowed, dtype: int64

In [8]:
df['cat_or_dog_allowed'].value_counts()

0    24787
1    22473
Name: cat_or_dog_allowed, dtype: int64

**Task 2:** Using your `pandas` and dataviz skills decide on two features that you want to engineer for your dataset. Next, modify your `wrangle` function to add those features. 

**Note:** You can learn more about feature engineering [here](https://en.wikipedia.org/wiki/Feature_engineering). Here are some ideas for new features:

- Does the apartment have a description?
- Length of description.
- Total number of perks that apartment has.
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths).

In [9]:
# Conduct your exploratory data analysis here, 
# and then modify the function above.

# price is influenced by pets allowed 
df['cats_allowed']
df['dogs_allowed']
df['pets_allowed'] = df['cats_allowed'] + df['dogs_allowed']
df['pets_allowed'].value_counts()

def checkPetNumber(cell_contents):
  if (cell_contents > 0):
    cell_contents = cell_contents - 1
  return cell_contents

df['pets_allowed'] = df['pets_allowed'].apply(checkPetNumber)
df['pets_allowed'].value_counts()


0    26336
1    20924
Name: pets_allowed, dtype: int64

In [10]:
df['cats_allowed']
df['dogs_allowed']
df['cat_or_dog_allowed'] = df['cats_allowed'] + df['dogs_allowed']
df['cat_or_dog_allowed'].value_counts()

def checkTruthValue(cell_contents):
  if(cell_contents > 0):
    cell_contents = 1
  return cell_contents

df['cat_or_dog_allowed'] = df['cat_or_dog_allowed'].apply(checkTruthValue)
df['cat_or_dog_allowed'].value_counts()

0    24787
1    22473
Name: cat_or_dog_allowed, dtype: int64

# II. Split Data

**Task 3:** Split your DataFrame `df` into a feature matrix `X` and the target vector `y`. You want to predict `'price'`.

**Note:** In contrast to the last module project, this time you should include _all_ the numerical features in your dataset.

In [11]:
X = df.drop(columns=['price'])
y = df['price']

X.shape
X.head()

,bathrooms,bedrooms,latitude,longitude,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,cat_and_dog_allowed,cat_or_dog_allowed,pets_allowed
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,40.7145,-73.9425,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-12 12:19:27,1.0,2,40.7947,-73.9667,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
2016-04-17 03:26:41,1.0,1,40.7388,-74.0018,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-18 02:22:02,1.0,1,40.7539,-73.9677,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-28 01:32:41,1.0,4,40.8241,-73.9493,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Task 4:** Split `X` and `y` into a training set (`X_train`, `y_train`) and a test set (`X_test`, `y_test`).

- Your training set should include data from April and May 2016. 
- Your test set should include data from June 2016.

In [12]:
# April = 4, May = 5
mask1 = X.index.month == 4
print(mask1)
mask2 = X.index.month == 5
print(mask2)

mask = mask1 + mask2
mask

[False False  True ...  True  True  True]
[False False False ... False False False]


array([False, False,  True, ...,  True,  True,  True])

In [13]:
X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

X_train.shape

(30806, 31)

In [14]:
X_test.shape

(16454, 31)

# III. Establish Baseline

**Task 5:** Since this is a **regression** problem, you need to calculate the baseline mean absolute error for your model. First, calculate the mean of `y_train`. Next, create a list `y_pred` that has the same length as `y_train` and where every item in the list is the mean. Finally, use `mean_absolute_error` to calculate your baseline.

In [15]:
y_pred = [y_train.mean()] * len(y_train)
baseline_mae = mean_absolute_error(y_train, y_pred)
print('Baseline MAE:', baseline_mae)

Baseline MAE: 1202.398300781848


# IV. Build Model

**Task 6:** Build and train a `LinearRegression` model named `model` using your feature matrix `X_train` and your target vector `y_train`.

In [16]:
# Step 1: Import predictor class


# Step 2: Instantiate predictor
model = LinearRegression()

# Step 3: Fit predictor on the (training) data
model.fit(X_train, y_train);

# V. Check Metrics

**Task 7:** Calculate the training and test mean absolute error for your model.

In [17]:
training_mae = mean_absolute_error(y_train, model.predict(X_train))
test_mae = mean_absolute_error(y_test, model.predict(X_test))

print('Training MAE:', training_mae)
print('Test MAE:', test_mae)

Training MAE: 692.5250790561875
Test MAE: 701.0860827674272


**Task 8:** Calculate the training and test $R^2$ score for your model.

In [21]:
training_r2 = mean_squared_error(y_train, model.predict(X_train), squared=False)
test_r2 = mean_squared_error(y_test, model.predict(X_test), squared=False)

print('Training RSME:', training_r2)
print('Test RSME:', test_r2)

Training RSME: 1092.6927859470782
Test RSME: 1080.2414072139372


In [23]:
print('Training R^2:', model.score(X_train, y_train))
print('Test R^2:', model.score(X_test, y_test))

Training R^2: 0.6163632890037458
Test R^2: 0.6262977038183297


# VI. Communicate Results

**Stretch Goal:** What are the three most influential coefficients in your linear model? You should consider the _absolute value_ of each coefficient, so that it doesn't matter if it's positive or negative.

In [19]:
intercept = model.intercept_
coeff = model.coef_

print('PRICE =', intercept, '+', coeff[-2], 'X CATS AND DOGS ALLOWED')

PRICE = -1070060.378830987 + 794818266133038.8 X CATS AND DOGS ALLOWED
